#Merge Predictions
This script merges two predictions. It has support for average or additative blending. Furtermore it can produce the final submission file for kaggle.

##Import & Setup

In [ ]:
import tensorflow as tf
import numpy as np

import os
import time
import glob
import datetime

from matplotlib import pyplot as plt
from IPython import display

from google.colab import drive


from skimage import io, util, filters
from skimage.filters import *
import cv2




### Mount Data Location
Mounts the Gdrive, where the raw prediction output of the models resides.

In [ ]:
drive.mount('/content/drive')
 
 
PATH = "/content/drive/My Drive/CIL Project Images"
print(PATH)

##Load Data
Load the satellite images and the predictions of the models that get combined.

In [ ]:
satellite_path =  PATH  + '/dataset/test_images/'
print("Loading: " + satellite_path)
satellite_images_list = [cv2.imread(file) for file in sorted(
glob.glob(satellite_path + '*.png'))]

pix2pix_path = PATH  + "/predictions/predictions_michi/pix2pix_snapshot/"
print("Loading: " + pix2pix_path)
pix2pix_preds = [cv2.imread(file, cv2.IMREAD_GRAYSCALE) for file in sorted(
glob.glob(pix2pix_path + '*.png'))]

unet_path =  PATH  + '/predictions/predictions_michi/dani/report_unet_dilated_5x5_augmented_snapshot_sgd/predictions/'
print("Loading: " + unet_path)
unet_preds = [cv2.imread(file, cv2.IMREAD_GRAYSCALE) for file in sorted(
glob.glob(unet_path + '*.png'))]

print(len(satellite_images_list))
print(len(pix2pix_preds))
print(len(unet_preds))


##Merge the predictions
Does average blending and additative blending and plots the results.

In [ ]:
add_blend = []
avg_blend = []
alpha = 0.5
beta = (1.0 - alpha)
for p2p, unet in zip(pix2pix_preds, unet_preds):
  add_merged = cv2.add(p2p, unet)
  avg_merged = cv2.addWeighted(p2p, alpha, unet, beta, 0.0)
  add_blend.append(add_merged)
  avg_blend.append(avg_merged)

for idx in range(len(satellite_images_list)):
  plt.figure(figsize=(25,25))

  display_list = [satellite_images_list[idx],pix2pix_preds[idx],unet_preds[idx],add_blend[idx],avg_blend[idx]]
 
  title = ['Input Image', 'pix2pix', 'UNet', 'Additative Blend', 'Average Blend']

  for i in range(len(display_list)):
    plt.subplot(1, len(display_list), i+1)
    plt.title(title[i], fontsize=24)
    plt.imshow(display_list[i], cmap="gray" )
    plt.axis('off')
  plt.show()


##Write blend to disk
This saves the blends to disk and also ouputs the final kaggle submission file. NOTE: Run the cell directly below first, as it includes the helper code to generate the submission file.

In [ ]:
avg_blend_path = PATH  + "/predictions/predictions_michi/merged/blend/"
add_blend_path = PATH  + "/predictions/predictions_michi/merged/add/"

#Get filenames
filenames = []
for file in glob.glob(pix2pix_path + "*.png"):
    filenames.append(os.path.basename(file))
filenames = sorted(filenames)

#write to disk
for idx in range(len(satellite_images_list)):
  img_avg = avg_blend[idx]
  img_add = add_blend[idx]
  img_add_path = add_blend_path + str(filenames[idx])
  img_avg_path = avg_blend_path + str(filenames[idx])
  print(img_add_path)
  print(img_avg_path)
  cv2.imwrite(img_add_path, img_add)
  cv2.imwrite(img_avg_path, img_avg)

#submit predictions
submit_predictions(filenames, avg_blend_path)
submit_predictions(filenames, add_blend_path)


In [ ]:
import re
import matplotlib.image as mpimg
import numpy as np
foreground_threshold = 0.35 # percentage of pixels > 1 required to assign a foreground label to a patch

# assign a label to a patch
def patch_to_label(patch):
    df = np.mean(patch)
    if df > foreground_threshold:
        return 1
    else:
        return 0

def mask_to_submission_strings(image_filename):
    """Reads a single image and outputs the strings that should go into the submission file"""
    img_number = int(re.search(r"\d+", image_filename).group(0))
    im = mpimg.imread(image_filename)
    patch_size = 16
    for j in range(0, im.shape[1], patch_size):
        for i in range(0, im.shape[0], patch_size):
            patch = im[i:i + patch_size, j:j + patch_size]
            label = patch_to_label(patch)
            yield("{:03d}_{}_{},{}".format(img_number, j, i, label))


def masks_to_submission(submission_filename, *image_filenames):
    """Converts images into a submission file"""
    with open(submission_filename, 'w') as f:
        f.write('id,prediction\n')
        for fn in image_filenames[0:]:
            f.writelines('{}\n'.format(s) for s in mask_to_submission_strings(fn))

#### Applies the "mask_to_submission file that converts the predicted images to our output format 
#   Output format: Each image is split into patches of 16 x 16 pixels, and then a 0 or 1 label is assigned to it 
#   based on our predicted pixel-wise label
#   The public test score is based on those patch-wise predictions 
#  ####

def submit_predictions(filenames, path, thresholding = False):
  # Path(path + "/results/csv").mkdir(parents=True, exist_ok=True)
  submission_filename = path + 'submission_' + datetime.datetime.now().strftime('%Y-%m-%d_%H:%M') + '.csv'
  image_filenames = []
  for i in range(0, 94):
    number = filenames[i]
    filename = path + number #blend or add
    # filename = path + number #old
    if not os.path.isfile(filename):
        print(filename + " not found")
        continue
    image_filenames.append(filename)
    
  masks_to_submission(submission_filename, *image_filenames)
